## TELCO CUSTOMER BASE (TCB) CHURN: An Initial Analysis

#### WORKING PREMISE:

**While each Customer will have their own particular reason for parting ways with Telco, the two key ingredients needed for Customer churn are:**
1. Low or No "Perception of Commitment" to Telco.
2. Minimal Switching Costs associated with leaving Telco.

#### The goals of this initial exploration are as follows:
- Discover the "Driving Factors" which contribute most to Customer churn.
- Discover the Customer segment which is most likely to churn.
- Discover the Customer segment which is most responsible for revenue.
- Identify those Customers who exhibit the characteristics of said target Customer segment.
- Create a Machine Learning Model which can accurately predict Customer churn with greater accuracy than baseline.


In [ ]:
# DS Libraries
import pandas as pd
import numpy as np
from scipy import stats

# Data Acquisition
from pydataset import data
import env
import acquire as acq
import prepare as prp

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# scikit learn submodules
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
# Let's Acquire and Prepare the DataFrame
# load telco via acquire.py (be sure to place acquire.py in same directory as this Notebook)
df = acq.new_telco_data()

In [ ]:
# Using the prep_telco function in prepare.py, we will drop unneeded columns and encode others
df = prp.prep_telco(df)

#### Initial questions and hypothesis:
#### What % of Telco Customer Base (TCB) has No "Perception of  Commitment" to Telco in combination with low "Switching Costs"?

- Do customers with **"Month-to-Month"** contracts churn out at a statistically significant greater rate than the overall population?
- Do customers with **"Paperless Billing"** enabled churn out at a statistically significant greater rate than the overall population?
- Do customers **without "Dependents"** churn out at a statistically significant greater rate than the overall population?
- Do customers who comprise the largest customer segment churn out at a statistically significant greater rate than the overall population?

#### HYPOTHESIS: Customers in these segmnents churn at greater rates than the overall Telco Customer Base (TCB)

In [ ]:
# Using prepare.py, split the Dataset into TRAIN(.56), VALIDATE(.24), and TEST(.20) subsets:
train, validate, test = prp.split_data(df,'churn')
# quick check of the numbers for each should be 3943-1691-1409
train.shape[0],validate.shape[0],test.shape[0]

#### Technical disclaimer...

- The original dataset contained 7043 records.  Due to the unique req's for building a ML model that actually functions properly with UNSEEN data, we have split the data into several subsets, for Training the model and for Testing the model.  
- Therefore, note that our univariate analyses will imply 3943 total records instead.  The conclusions gained from the analysis will be just as valid as if we had used the full dataset.

### Let us now compare Customer Counts in a few specific Categories:

- **70%** of TCB has **NO DEPENDENTS** on Telco Data/Voice Plans  
- **59%** of TCB has **PAPERLESS BILLING** on Telco Data/Voice Plans 
- **54%** of TCB has **MONTH-to-MONTH** contracts on Telco Data/Voice Plans  

### Taken in combination, this represents 22.3 % of TCB

### What are the total counts of TCB by DEPENDENTS status?

In [ ]:
# Chart showing raw counts of customers with dependents and customers without:
print(f'TCB with NO DEPENDENTS:')
sns.countplot(data=train, x='dependents')
plt.show()
print(
    pd.concat([train['dependents'].value_counts(),
    train['dependents'].value_counts(normalize=True)],
         axis=1))

### TAKEAWAY:  70% of TCB does not have to consider the Switching Costs associated with transitioning DEPENDENTS to a new service provider.

### What are the total counts of TCB by PAPERLESS BILLING status?

In [ ]:
# Chart showing raw counts of customers with paperless_billing and customers without:
print(f'TCB with PAPERLESS BILLING:')
sns.countplot(data=train, x='paperless_billing')
plt.show()
print(
    pd.concat([train['paperless_billing'].value_counts(),
    train['paperless_billing'].value_counts(normalize=True)],
         axis=1))

### TAKEAWAY:  59% of TCB never sees a monthly paper bill.  This may reduce some Customers "Perception of Commitment"...  Out of Sight, Out of Mind...

### What are the total counts of TCB by CONTRACT TYPE?

In [ ]:
# Chart showing raw counts of customers by contract type:
print(f'TCB with MONTH-to_MONTH contract:')
sns.countplot(data=train, x='contract_type')
plt.show()
print(
    pd.concat([train['contract_type'].value_counts(),
    train['contract_type'].value_counts(normalize=True)],
         axis=1))

### TAKEAWAY:  54% of TCB does not have to consider monetary Switching Costs associated with breaking an extended contract.  Instead, they can churn-out at their convenience.

# BOTTOM LINE  \#1: 1 in 5 Customers exhibit ALL of these characteristics and an unmeasured plurality of Customers fall into at least one of these categories.

--------
--------
--------

### Let us now compare Customer Churn Rates in a few specific Categories and compare them to the overall TCB mean average:

In [ ]:
# Chart showing churn rate of customers by dependent status, as compared to overall churn rate:
sns.barplot(data=train,
            x='dependents',
            y = 'churn_Yes',
           ci=False)
plt.title(f'Customer CHURN Rate based on DEPENDENTS')
plt.axhline(train.churn_Yes.mean(),label='Overall TCB Churn Rate: ~ 26%',color='red',linewidth=4)
plt.legend()
plt.show()

### TAKEAWAY:  With the Overall TCB Churn Rate at ~ 26%, Customers WITHOUT DEPENDENTS churn-out at a sample rate of ~ 31% .

### This chart is nice, however how can we be sure that these results are not just due to random sampling?  How statistically significant is this result?
#### Glad you asked, let's run a quick Chi$^2$ test for Independence to find out how likely it is that we would have obtained these results via random sampling...

In [ ]:
# Lets Run a Chi2 Test to find out how likely it is that we would get this result by random sampling:

# H0: No statistically significant difference exists between the churn rates of customers with DEPENDENTS 
#     and ALL customers. ie the two samples were drawn from the same population.
    
# H1: A statistically significant difference exists between the churn rates of customers with DEPENDENTS 
#     and ALL customers.  ie the two samples were drawn from differing populations.

df_dependents_x =pd.crosstab(train['dependents'],train['churn'])
df_dependents_x
stats.chi2_contingency(df_dependents_x)

# This function has 4 outputs: Chi2 Coefficient (94.41...), p-value (2.55 e-22)(think...very close to ZERO)),
# the number of Degrees of Freedom (1) and a 2-dim array showing the expected frequency of each case assuming that
# null hypothesis were True
# With a p-value < 0.05, we can REJECT the NULL hypothesis.  There is evidence to suggest that a statistically
# significant difference exists between the two rates.

### TAKEAWAY:  With a p-value < 0.05, we can REJECT the NULL hypothesis.  There is evidence to suggest that a statistically significant difference exists between the two rates.

In [ ]:
# Chart showing churn rate of customers by paperless status, as compared to overall churn rate:
sns.barplot(data=train,
            x='paperless_billing',
            y = 'churn_Yes',
           ci=False)
plt.title(f'Customer CHURN Rate based on PAPERLESS')
plt.axhline(train.churn_Yes.mean(),label='Overall TCB Churn Rate: ~ 26%',color='red',linewidth=4)
plt.legend()
plt.show()

### TAKEAWAY:  With the Overall TCB Churn Rate at ~ 26%, Customers WITH PAPERLESS BILLING churn-out at a sample rate of ~ 33% .

### Again, how statistically significant is this result?
#### Let's run a quick Chi$^2$ test for Independence to find out how likely it is that we would have obtained these results via random sampling...

In [ ]:
# Lets Run a Chi2 Test to find out how likely it is that we would get this result by random sampling:

# H0: No statistically significant difference exists between the churn rates of customers with DEPENDENTS 
#     and ALL customers. ie the two samples were drawn from the same population.
    
# H1: A statistically significant difference exists between the churn rates of customers with DEPENDENTS 
#     and ALL customers.  ie the two samples were drawn from differing populations.

df_dependents_x =pd.crosstab(train['paperless_billing'],train['churn'])
df_dependents_x
stats.chi2_contingency(df_dependents_x)

# This function has 4 outputs: Chi2 Coefficient (131.66...), p-value (1.77 e-30)(think...very close to ZERO)),
# the number of Degrees of Freedom (1) and a 2-dim array showing the expected frequency of each case assuming that
# null hypothesis were True
# With a p-value < 0.05, we can REJECT the NULL hypothesis.  There is evidence to suggest that a statistically
# significant difference exists between the two rates.

### TAKEAWAY:  With a p-value < 0.05, we can REJECT the NULL hypothesis.  There is evidence to suggest that a statistically significant difference exists between the two rates.

In [ ]:
# Chart showing churn rate of customers by contract_type, as compared to overall churn rate:
sns.barplot(data=train,
            x='contract_type',
            y = 'churn_Yes',
           ci=False)
plt.title(f'Customer CHURN Rate based on CONTRACT TYPE')
plt.axhline(train.churn_Yes.mean(),label='Overall TCB Churn Rate: ~ 26%',color='red',linewidth=4)
plt.legend()
plt.show()

### TAKEAWAY:  With the Overall TCB Churn Rate at ~ 26%, Customers with MONTH-TO-MONTH contracts churn-out at a sample rate of ~ 43% .

### How statistically significant is this result?
#### A Chi$^2$ test for Independence to find out how likely it is that we would have obtained these results via random sampling...

In [ ]:
# Lets Run a Chi2 Test to find out how likely it is that we would get this result by random sampling:

# H0: No statistically significant difference exists between the churn rates of customers with DEPENDENTS 
#     and ALL customers. ie the two samples were drawn from the same population.
    
# H1: A statistically significant difference exists between the churn rates of customers with DEPENDENTS 
#     and ALL customers.  ie the two samples were drawn from differing populations.

df_dependents_x =pd.crosstab(train['contract_type'],train['churn'])
df_dependents_x
stats.chi2_contingency(df_dependents_x)

# This function has 4 outputs: Chi2 Coefficient (650.61...), p-value (2.25 e-142)(think...very close to ZERO)),
# the number of Degrees of Freedom (2) and a 2-dim array showing the expected frequency of each case assuming that
# null hypothesis were True
# With a p-value < 0.05, we can REJECT the NULL hypothesis.  There is evidence to suggest that a statistically
# significant difference exists between the two rates.

### TAKEAWAY:  With a p-value < 0.05, we can REJECT the NULL hypothesis.  There is evidence to suggest that a statistically significant difference exists between the two rates.

# BOTTOM LINE  \#2: ALL Customers who exist in these categories exhibit a CHURN Rate Greater than the TCB mean average rate.

# 1 + 2 = ...
# This situation must be addressed...the largest segment of our TCB also Churns out at the greatest rate.

----
----
----

In [ ]:
# We divide the columns into categorical, categorical_encoded, and numerical

cat_cols, cat_cols_e, num_cols = [], [], []
for col in train.columns:
    if train[col].dtype == 'O':
        cat_cols.append(col)
    else:
        if train[col].nunique() < 10:
            cat_cols_e.append(col)
        else:
            num_cols.append(col)
            
# We then assign the needed columns to variables and then affix those column groupings to the appropriate
# TRAIN, VALIDATE, and TEST subsets
X_cols = cat_cols_e
y_cols = 'churn_Yes'
X_cols.remove('churn_Yes')
X_cols,y_cols

X_train = train[X_cols]
y_train = train['churn']

X_validate = validate[X_cols]
y_validate = validate['churn']

X_test = test[X_cols]
y_test = test['churn']

# create the Decision Tree Classifier model 
clf = DecisionTreeClassifier()
# fit the model to the TRAIN dataset:
clf.fit(X_train, y_train)
# use the model by calling for the predictions made via the TRAIN dataset
model_a_preds = clf.predict(X_train)
pd.crosstab(model_a_preds,y_train) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

### GIVEN THE FOLLOWING:
- Customizable options provided by Telco is the reason we have our Customer Base.
- Removing Month-to-Month contracts or any other variable which may appear to be correlated with high churn rates is "bad business" and simply not feasible.
- Accepting significantly higher Churn rates in certain Customer Segments as just "the cost of doing business" is not an appropriately proactive approach.
- Providing "loyalty incentives" to all customers regardless of their propensity to Churnis not economically viable for long-term growth.

# BOTTOM LINE  \#3: We must find a way to IDENTIFY those Customers who are most likely to Churn and PROVIDE Targeted "Loyalty Incentives" designed for Customer retention. 

## The good news is that we have a prototype Model that is being tested exactly for this puprose!

# 1 + 2 = 3

----
----
----

### LET"S FIRST ESTABLISH OUR BASELINE:
- Current Overall TCB Churn rate is ~26.5%.
- Therefore, 73.5% of TCB has no current propensity to churn. CALL THIS BASELINE ACCURACY (73.5%)
- If we assume all Customers Will Not Churn...we will be wrong in 26.5% of all cases.
- This is the current state of affairs at Telco, and we can easily outperform this baseline. 


### MODEL A (codename: Vanilla_Tree) :

- Model TRAIN ACCURACY: 81%
- Model VALIDATE ACCURACY: 77% slighly OVERFIT
- RECALL for YES: 55%

In [ ]:
# here is a classification report with many different measurements and ratios based upon the confusion matrix
# for our purposes here we will look at ACCURACY (0.81) and RECALL for the YES class (0.55).

print(
    classification_report(y_train,
                      model_a_preds))

In [ ]:
# Here we will evaluate the difference between the ACCURACY score on TRAIN datatset and compare it to the ACCURACY
# score on VALIDATE.  If the difference between the two accuracy scores is aproaching 10, then this is an
# indication that our model may be OVERFIT...w/o explaining the concept, this is not what we want.  We want a model
# that will perform consistently on both TRAIN and VALIDATE.

print(f'''
Performance in accuracy of Decision Tree 1 on training data:
Accuracy (train): {clf.score(X_train, y_train)}
Accuracy (validate): {round(clf.score(X_validate, y_validate), 5)}''')

### MODEL B (codename: Barry_Bonds) :

- Model ACCURACY: 81% 
- Model VALIDATE ACCURACY: 77% slighly OVERFIT
- RECALL for YES: 59%

In [ ]:
# create the Random Forest model 
rf1 = RandomForestClassifier(n_estimators=25)
# fit the model to the TRAIN dataset:
rf1.fit(X_train, y_train)
# use the model by calling for the predictions made via the TRAIN dataset
model_b_preds = rf1.predict(X_train)
pd.crosstab(model_b_preds,y_train) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

In [ ]:
# here is a classification report with many different measurements and ratios based upon the confusion matrix
# for our purposes here we will look at ACCURACY (0.81) and RECALL for the YES class (0.59).

print(
    classification_report(y_train,
                      model_b_preds))

In [ ]:
# Here we will evaluate the difference between the ACCURACY score on TRAIN datatset and compare it to the ACCURACY
# score on VALIDATE.  If the difference between the two accuracy scores is aproaching 10, then this is an
# indication that our model may be OVERFIT...w/o explaining the concept, this is not what we want.  We want a model
# that will perform consistently on both TRAIN and VALIDATE.

print(f'''
Performance in accuracy of Decision Tree 1 on training data:
Accuracy (train): {rf1.score(X_train, y_train)}
Accuracy (validate): {round(rf1.score(X_validate, y_validate), 5)}''')

### MODEL C (codename: El_Camino) :


- Model ACCURACY: 81% 
- Model VALIDATE ACCURACY: 77% slighly OVERFIT
- RECALL for YES: 58%

In [ ]:
# create the Random Forest model 
rf2 = RandomForestClassifier(n_estimators=101)
# fit the model to the TRAIN dataset:
rf2.fit(X_train, y_train)
# use the model by calling for the predictions made via the TRAIN dataset
model_c_preds = rf2.predict(X_train)
pd.crosstab(model_c_preds,y_train) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

In [ ]:
# here is a classification report with many different measurements and ratios based upon the confusion matrix
# for our purposes here we will look at ACCURACY (0.81) and RECALL for the YES class (0.58).

print(
    classification_report(y_train,
                      model_c_preds))

In [ ]:
# Here we will evaluate the difference between the ACCURACY score on TRAIN datatset and compare it to the ACCURACY
# score on VALIDATE.  If the difference between the two accuracy scores is aproaching 10, then this is an
# indication that our model may be OVERFIT...w/o explaining the concept, this is not what we want.  We want a model
# that will perform consistently on both TRAIN and VALIDATE.

print(f'''
Performance in accuracy of Decision Tree 1 on training data:
Accuracy (train): {rf2.score(X_train, y_train)}
Accuracy (validate): {round(rf2.score(X_validate, y_validate), 5)}''')

----
----
----
## IN REVIEW:

### MODEL A (codename: Vanilla_Tree) :

- Model TRAIN ACCURACY: 81%
- Model VALIDATE ACCURACY: 77% slighly OVERFIT
- RECALL for YES: 55%

### MODEL B (codename: Barry_Bonds) :

- Model ACCURACY: 81% 
- Model VALIDATE ACCURACY: 77% slighly OVERFIT
- RECALL for YES: 59%

### MODEL C (codename: El_Camino) :


- Model ACCURACY: 81% 
- Model VALIDATE ACCURACY: 77% slighly OVERFIT
- RECALL for YES: 58%

## Currently, the best option is Barry_Bonds due to the Optimized Recall for Yes
### This model will identify the LARGEST percentage of Customers with a propensity to Churn at 59% (Hit Rate).  Currently, with a Hit-Rate of 0%, we do not have the option to make any offers to Customers who are at-risk for Churn.  Another way to say it is that we MISS on 100% of all customers who will CHURN.
### Barry's Hit-Rate will be able to successfully ID 59% of these Customers and we will actually be in position to make targeted "Loyalty Offers" to 59% of this segment of interest.
### Given the fact that Barry is an unsophisticated model, in a prototype phase, we will only find BETTER models in the future!

In [ ]:
barry_preds = rf1.predict(X_test)
barry_proba = rf1.predict_proba(X_test)
pd.crosstab(barry_preds,y_test) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

In [ ]:
# here is a classification report with many different measurements and ratios based upon the TEST subset
# for our purposes here we will look at ACCURACY (0.76) and RECALL for the YES class (0.45).

print(
    classification_report(y_test,
                      barry_preds))

In [ ]:
pd.crosstab(barry_preds,y_test) # a confusion matrix with ACTUALS as columns and PREDICTIONS as rows

In [ ]:
barry_proba

In [ ]:
# create array of probabilities of being versicolor (versicolor == 1)

x_barry_proba = np.array([i[0] for i in barry_proba])
y_barry_proba = np.array([i[1] for i in barry_proba])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

# scatter plot where x is the probabilities and y is the class (0, 1)
ax.scatter(y_barry_proba,y_barry_proba,alpha=0.1,color='red',edgecolors='black',s=50)